In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import SGD
from keras.utils import to_categorical
import matplotlib.pyplot as plt

def load_and_prepare_data(data_path):
    karakterler_resim = []
    karakterler_etiket = []

    karakterler = os.listdir(data_path)

    for karakter in karakterler:
        char_path = os.path.join(data_path, karakter)

        for file in os.listdir(char_path):
            img_path = os.path.join(char_path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (28, 28))
            img = np.reshape(img, (28 * 28))
            karakterler_resim.append(img)
            karakterler_etiket.append(karakter)

    X = np.array(karakterler_resim)
    y = np.array(karakterler_etiket)

    # Etiketleri LabelEncoder kullanarak encode etme
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Etiketleri one-hot encoding'e dönüştürme
    y_encoded = to_categorical(y_encoded, num_classes=len(np.unique(y)))

    # Veriyi karıştırma
    X_shuffle, y_shuffle = shuffle(X, y_encoded, random_state=36)

    # Veriyi eğitim ve doğrulama setlerine ayırma
    X_train, X_val, y_train, y_val = train_test_split(X_shuffle, y_shuffle, test_size=0.2, random_state=42)

    return X_train, X_val, y_train, y_val, label_encoder

def create_and_train_model(X_train, y_train, X_val, y_val):
    model = Sequential()
    model.add(Dense(units=784, activation='sigmoid', input_dim=28 * 28))
    model.add(Dense(units=392, activation='sigmoid'))
    model.add(Dense(units=36, activation='softmax'))

    model.compile(optimizer=SGD(learning_rate=0.02), loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=10)

    return model, history

def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.plot(history.history['loss'], label='Eğitim Kaybı')
    plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
    plt.xlabel('Epochs')
    plt.ylabel('Kayıp')
    plt.title('Kayıp Grafiği')
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
    plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
    plt.xlabel('Epochs')
    plt.ylabel('Doğruluk')
    plt.title('Doğruluk Grafiği')
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
    plt.xlabel('Epochs')
    plt.ylabel('Doğrulama Kaybı')
    plt.title('Doğrulama Kaybı Grafiği')
    plt.legend()

    plt.tight_layout()
    plt.show()

def save_model(model, model_path='model.h5'):
    model.save(model_path)

def load_and_process_image(image_path, contours_data="a"):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, threshold1=30, threshold2=100)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    i = 1
    folder_path = os.path.join(os.getcwd(), contours_data)
    os.makedirs(folder_path, exist_ok=True)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        character = image[y:y + h, x:x + w]

        if y + h - y > 15 and x + w - x > 3:
            cv2.imwrite(os.path.join(folder_path, f"{i}.png"), character)
            i += 1

    cv2.imwrite("Edges.jpg", edges)

def predict_with_model(model, folder_path):
    predictions = []

    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".png"):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (28, 28))
            img = np.reshape(img, (28 * 28))
            img = np.reshape(img, (1, 784, 1))
            img = img / 255.0
            prediction = model.predict(img)
            predictions.append(prediction)

    predictions = np.array(predictions)
    predicted_classes = np.argmax(predictions, axis=1)

    return predictions  # Return the predictions list

# Veri setini yükleme ve modeli eğitme
X_train, X_val, y_train, y_val, label_encoder = load_and_prepare_data('/kaggle/input/data')
model, history = create_and_train_model(X_train, y_train, X_val, y_val)

# Eğitim sürecini görselleştirme
plot_training_history(history)

# Modeli kaydetme
save_model(model, 'model.h5')

# El yazısı içeren görüntüyü işleme ve tahmin
load_and_process_image("/kaggle/input/png-elyazisi/yz/1.jpg")
predictions = predict_with_model(model, 'a')  # Use the returned predictions

# Tahmin edilen sınıfları etiketlere dönüştürme
predicted_classes = np.argmax(predictions, axis=2)
predicted_labels = label_encoder.inverse_transform(predicted_classes)
print(predicted_labels)